In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import os


In [2]:
def _fix_mojibake(s: str) -> str:
    if not s:
        return s
    if 'Ã' in s or 'Â' in s:
        for enc in ('latin1', 'cp1252'):
            try:
                fixed = s.encode(enc).decode('utf-8')
                if (fixed.count('Ã') + fixed.count('Â')) < (s.count('Ã') + s.count('Â')):
                    return fixed
                return fixed
            except Exception:
                continue
    return s


def inicializar_banco(db_path="projeto_fernando_pessoa/textos.db"):
    criar = not os.path.exists(db_path)
    conn = sqlite3.connect(db_path)
    if criar:
        cursor = conn.cursor()
        cursor.execute(
            """
            CREATE TABLE textos (
                id INTEGER PRIMARY KEY,
                autor TEXT,
                titulo TEXT,
                texto TEXT
            )
            """
        )
        conn.commit()
    return conn


def ja_existe(conn, texto_id):
    cursor = conn.cursor()
    cursor.execute("SELECT 1 FROM textos WHERE id=?", (texto_id,))
    return cursor.fetchone() is not None


def salvar_texto(conn, texto_id, autor, titulo, texto):
    cursor = conn.cursor()
    cursor.execute(
        "INSERT OR REPLACE INTO textos (id, autor, titulo, texto) VALUES (?, ?, ?, ?)",
        (texto_id, autor, titulo, texto),
    )
    conn.commit()


def buscar_textos_por_autores(autores: list, inicio: int = 1, fim: int = 5000, db_path="textos.db"):
    base_url = "http://arquivopessoa.net/textos/{}"
    resultados = {autor: {} for autor in autores}

    conn = inicializar_banco(db_path)

    for i in range(inicio, fim + 1):
        if ja_existe(conn, i):
            print(f"ID {i} já existe no banco, pulando...")
            continue

        url = base_url.format(i)
        try:
            response = requests.get(url, timeout=8)
            if response.status_code != 200:
                continue

            soup = BeautifulSoup(response.content, "html.parser")

            autor_div = soup.find("div", class_="autor")
            if not autor_div:
                continue

            autor_encontrado = _fix_mojibake(autor_div.get_text(strip=True))

            if autor_encontrado in resultados:
                titulo_tag = soup.find("h1", class_="titulo-texto")
                titulo = _fix_mojibake(titulo_tag.get_text(strip=True) if titulo_tag else "Sem título")

                texto_div = soup.find("div", class_="texto-poesia")
                texto = _fix_mojibake(texto_div.get_text("\n", strip=True) if texto_div else "Sem texto")

                resultados[autor_encontrado][i] = [titulo, texto]
                salvar_texto(conn, i, autor_encontrado, titulo, texto)
                print(f"Autor '{autor_encontrado}' encontrado no ID {i} -> {titulo}")

        except requests.RequestException as e:
            print(f"Erro ao acessar {url}: {e}")

    conn.close()
    return resultados

In [25]:

if __name__ == "__main__":
    autores = ["Alberto Caeiro", "Álvaro de Campos", "Ricardo Reis", "Bernardo Soares"]
    resultados = buscar_textos_por_autores(autores, 4995, 5000)

    print("\nResumo:")
    for autor, textos in resultados.items():
        print(f"\n{autor}: {len(textos)} textos")
        for id_texto, dados in textos.items():
            print(f"  {id_texto}: {dados[0]}")


Resumo:

Alberto Caeiro: 0 textos

Álvaro de Campos: 0 textos

Ricardo Reis: 0 textos

Bernardo Soares: 0 textos
